<table style="width:100%; border-collapse: collapse;">
  <tr>
    <td style="width:20%; vertical-align:middle;">
      <img src="LogoUVG.png" width="400"/>
    </td>
    <td style="text-align:left; vertical-align:middle;">
      <h2 style="margin-bottom: 0;">Universidad del Valle de Guatemala - UVG</h2>
      <h3 style="margin-top: 0;">Facultad de Ingeniería - Ciencia de la Computación</h3>
      <p style="font-size: 16px; margin-bottom: 0; margin-top: -20px">
        <strong>Course:</strong> CC3106 - Responsible AI
        <strong>Section:</strong> 10
      </p>
      <p style="font-size: 16px; margin: 0;"><strong>Laboratorio 2:</strong> Interpretando Vectores
</p>
      <br>
      <p style="font-size: 15px; margin: 0;"><strong>Autor:</strong></p>
      <ul style="margin-top: 5px; padding-left: 20px; font-size: 15px;">
        <li>Diego Alexander Hernández Silvestre - <strong>21270</strong></li>
      </ul>
    </td>
  </tr>
</table>


# ================================================
# Laboratorio 2 — Similitud de Coseno
# ================================================

# Instrucciones
1. **Carga de embeddings**  
   - Utilizar las representaciones vectoriales preentrenadas (*GloVe 100d*).  
   - Verificar que los términos requeridos estén contenidos en el vocabulario.  

2. **Cálculo de vectores de expresión**  
   - Construir expresiones vectoriales simples mediante operaciones aritméticas (suma, resta).  

3. **Medición de similitud**  
   - Calcular la similitud de coseno entre los vectores de la expresión y los vectores de referencia.  
   - Reportar:  
     - Valor de la similitud de coseno.  
     - Normas de cada vector (\(\|\mathbf{u}\|\)).  

4. **Casos de estudio**  
   Realizar caso por caso

5. **Documentación de resultados**  
   - Registrar en el notebook los valores obtenidos de similitud y normas vectoriales para cada caso.  
   - Incluir comentarios breves interpretando los resultados.  

6. **Conclusiones**  
   - Redactar conclusiones **caso por caso** sobre la utilidad de la similitud de coseno.

In [25]:
# Solo si es necesario
# !pip install gensim

In [26]:
import numpy as np
from typing import List
import gensim.downloader as api

In [27]:
# ========= 1) Cargar GloVe 100D =========
model = api.load("glove-wiki-gigaword-100")

In [28]:
# ========= 2) Funciones utilitarias =========
def embedding(word: str) -> np.ndarray:
    """Vector de la palabra (lanza KeyError si no está en vocab)."""
    return model[word]

def build_vector(plus: List[str], minus: List[str]) -> np.ndarray:
    """Suma y resta embeddings explícitamente."""
    dim = model[next(iter(model.key_to_index))].shape[0]
    v = np.zeros(dim, dtype=np.float32)
    for w in plus:
        if w in model:
            v += model[w]
        else:
            print(f"[OOV] '{w}' no está en el vocabulario.")
    for w in minus:
        if w in model:
            v -= model[w]
        else:
            print(f"[OOV] '{w}' no está en el vocabulario.")
    return v

def cos(a: np.ndarray, b: np.ndarray) -> float:
    na, nb = np.linalg.norm(a), np.linalg.norm(b)
    if na == 0 or nb == 0:
        return float("nan")
    return float(np.dot(a, b) / (na * nb))

def vec_norm(v: np.ndarray) -> float:
    return float(np.linalg.norm(v))

def report_case(label, plus, minus, target_plus):
    expr_vec   = build_vector(plus, minus)
    target_vec = build_vector(target_plus, [])
    print("====", label, "====")
    print("Expresión =", " + ".join(plus) + " - " + " - ".join(minus) if minus else " + ".join(plus))
    print("Target    =", " + ".join(target_plus))
    print("cos(expr, target) =", round(cos(expr_vec, target_vec), 6))
    print("||expr||           =", round(vec_norm(expr_vec), 6))
    print("||target||         =", round(vec_norm(target_vec), 6))
    print()

def nearest_neighbors(plus: List[str], minus: List[str], topn: int = 3):

    expr_vec = build_vector(plus, minus)
    expr_norm = np.linalg.norm(expr_vec)
    if expr_norm == 0 or not np.isfinite(expr_norm):
        return []

    plus_set, minus_set = set(plus), set(minus)
    dists = []
    for word in model.index_to_key:
        if word in plus_set or word in minus_set:
            continue
        vec = model[word]
        sim = cos(expr_vec, vec)
        dists.append((word, sim))

    dists.sort(key=lambda x: x[1], reverse=True)
    return dists[:topn]

## Caso 1 — Capitales

Construya un vector que represente la relación:  
**paris + italy - france ≈ rome**

1. Defina el vector de la expresión (`expr_vec`).  
2. Defina el vector objetivo (`target_vec`).  
3. Calcule `cos(expr, target)`, `||expr||` y `||target||`.  
4. Escriba su conclusión.  

In [29]:
report_case("Caso 1 — Capitales", plus=["paris", "italy"], minus=["france"], target_plus=["rome"])

==== Caso 1 — Capitales ====
Expresión = paris + italy - france
Target    = rome
cos(expr, target) = 0.8084
||expr||           = 6.227801
||target||         = 5.523363



El coseno entre la expresión paris + italy - france y el vector rome fue de 0.8084, lo cual una alta similitud semántica. Esto confirma que el modelo GloVe captura correctamente la relación que existe entre capitales: París es a Francia como Roma es a Italia.

## Caso 2 — Dictador/País

Construya un vector que represente la relación:  
**hitler + italy - germany**

1. Construya el vector de la expresión.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Escriba su interpretación de los resultados.  

In [30]:
plus = ["hitler", "italy"]
minus = ["germany"]
expr_vec = build_vector(plus, minus)
print("==== Caso 2 — Dictador/País ====")
print("Expresión =", (" + ".join(plus) + " - " + " - ".join(minus)) if minus else (" + ".join(plus)))
print("||expr|| =", round(vec_norm(expr_vec), 6))
print()
vecinos = nearest_neighbors(plus, minus, topn=3)
print("3 vecinos más cercanos y sus similitudes:")
for i, (w, sim) in enumerate(vecinos, 1):
    print(f"  {i}. {w:20s}  {sim:.6f}")

==== Caso 2 — Dictador/País ====
Expresión = hitler + italy - germany
||expr|| = 6.33357

3 vecinos más cercanos y sus similitudes:
  1. mussolini             0.816139
  2. fascist               0.668780
  3. stalin                0.639061


El vector resultante hitler + italy - germany mostró la mayor similitud coseno con mussolini (0.816), seguido de fascist y stalin. Esto indica que el modelo GloVe captura correctamente la asociación entre líderes dictatoriales y sus respectivos países o ideologías. En este caso, la expresión logra representar el contexto de Hitler-Alemania hacia Italia, surgiendo de esta forma, Mussolini como vecino principal.

## Caso 3 — Religión/Líder

Construya un vector que represente la relación:  
**christianity ≈ jesus**

1. Construya el vector de la palabra `christianity`.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Escriba su interpretación: ¿aparece `jesus` entre los vecinos?  

In [31]:
plus = ["christianity"]
minus = []

expr_vec = build_vector(plus, minus)
print("==== Caso 3 — Religión/Líder ====")
print("Expresión =", (" + ".join(plus) + " - " + " - ".join(minus)) if minus else (" + ".join(plus)))
print("||expr|| =", round(vec_norm(expr_vec), 6))
print()

vecinos = nearest_neighbors(plus, minus, topn=3)
print("3 vecinos más cercanos y sus similitudes:")
for i, (w, sim) in enumerate(vecinos, 1):
    print(f"  {i}. {w:20s}  {sim:.6f}")

jesus_sim = cos(expr_vec, embedding("jesus"))
print(f"\nSimilitud coseno con 'jesus': {jesus_sim:.6f}")

==== Caso 3 — Religión/Líder ====
Expresión = christianity
||expr|| = 5.906148

3 vecinos más cercanos y sus similitudes:
  1. catholicism           0.875112
  2. protestantism         0.811677
  3. religion              0.781869

Similitud coseno con 'jesus': 0.542634


El vector de christianity se acerca fuertemente al concepto general de religión, más que a la figura histórica específica (jesus). La similitud con jesus (0.543) es positiva pero no está dentro de los vecinos más cercanos, lo que sugiere que, en GloVe 100d, christianity se representa más como categoría (catolicismo) que como líder fundador.

## Caso 4 — Opuestos

Analice la relación:  
**good - bad**

1. Construya el vector de la expresión `good - bad`.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Escriba su conclusión sobre si los vecinos reflejan un contraste u oposición semántica.

In [32]:
plus = ["good"]
minus = ["bad"]

expr_vec = build_vector(plus, minus)
print("==== Caso 4 — Opuestos ====")
print("Expresión =", (" + ".join(plus) + " - " + " - ".join(minus)) if minus else (" + ".join(plus)))
print("||expr|| =", round(vec_norm(expr_vec), 6))
print()

vecinos = nearest_neighbors(plus, minus, topn=3)
print("3 vecinos más cercanos y sus similitudes:")
for i, (w, sim) in enumerate(vecinos, 1):
    print(f"  {i}. {w:20s}  {sim:.6f}")

==== Caso 4 — Opuestos ====
Expresión = good - bad
||expr|| = 3.815561

3 vecinos más cercanos y sus similitudes:
  1. excellent             0.522756
  2. versatile             0.459313
  3. ideal                 0.457311


Los vecinos recuperados no reflejan una oposición directa (good vs. bad), sino un desplazamiento hacia el polo positivo del espacio semántico. Términos como excellent e ideal muestran que la operación good - bad se centra en cualidades positivas. En consecuencia, más que implicar un contraste, el resultado evidencia orientación hacia lo positivo.

## Caso 5 — Propio

Defina usted mismo un caso interesante. Ejemplos:  
- `king - man + woman`  
- `tokyo + france - japan`  
- `apple - technology`  

1. Construya la expresión vectorial que haya definido.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Redacte una conclusión explicando si los resultados corresponden a su hipótesis inicial.

In [33]:
plus = ["google","government"]
minus = ["attention"]
target = ["control"]

report_case("google + government - attention ≈ control", plus, minus, target)

expr_vec = build_vector(plus, minus)
vecinos = nearest_neighbors(plus, minus, topn=3)

print("3 vecinos más cercanos y sus similitudes:")
for i, (w, sim) in enumerate(vecinos, 1):
    print(f"  {i}. {w:20s}  {sim:.6f}")

sim_control = cos(expr_vec, embedding("control"))
print(f"\nSimilitud coseno con 'control': {sim_control:.6f}")


==== google + government - attention ≈ control ====
Expresión = google + government - attention
Target    = control
cos(expr, target) = 0.435727
||expr||           = 8.267729
||target||         = 5.697279

3 vecinos más cercanos y sus similitudes:
  1. microsoft             0.635188
  2. yahoo                 0.601885
  3. software              0.588352

Similitud coseno con 'control': 0.435727


Los vecinos que aparecieron son principalmente compañías o temas relacionados con tecnología, y muestran una similitud alta con la expresión. Esto da a entender que al combinar google y government—y restar attention—el vector se mueve hacia un significado más ligado al ámbito tecnológico y empresarial. La similitud con control (0.436) es moderada y apoya en parte la hipótesis de que existe una asociación con nociones de control o regulación en este contexto. Sin embargo, el modelo parece darle prioridad a entidades similares a google (como microsoft o yahoo) antes que a ideas más abstractas. De esta forma, los resultados apoyan parcialmente la hipótesis: captan cierta relación con el control y la regulación, pero destacan más la cercanía con otras compañías tecnológicas que con conceptos.